# Info
References https://www.kaggle.com/code/batu35/collaborative-filtering-using-movielens-dataset

In [1]:
# importing libraries
import pandas as pd
import numpy as np
from numpy import ndarray
from pandas import DataFrame
from sklearn.metrics.pairwise import pairwise_distances


In [29]:
movies=pd.read_csv("data/ml-100k/u.item",sep="|",encoding="latin-1",header=None) #Read movies
# We just need movie_id and movie_name
movies=movies.iloc[:,0:2]
movies.columns=["movie_id","movie_name"]
movies.head()

,movie_id,movie_name
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [30]:
rating_columns = ["user_id", "movie_id", "rating", "unix_timestamp"]
ratings: DataFrame = pd.read_csv("data/ml-100k/u.data", sep="\t", names=rating_columns, encoding="latin-1")
ratings=pd.merge(ratings,movies,on="movie_id",how="inner")
ratings.head()

,user_id,movie_id,rating,unix_timestamp,movie_name
0,196,242,3,881250949,Kolya (1996)
1,186,302,3,891717742,L.A. Confidential (1997)
2,22,377,1,878887116,Heavyweights (1994)
3,244,51,2,880606923,Legends of the Fall (1994)
4,166,346,1,886397596,Jackie Brown (1997)


In [3]:
# Ratings Data
print("\nRatings Data :")
print("shape : ", ratings.shape)
ratings.head()


Ratings Data :
shape :  (100000, 4)


,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [15]:
number_of_users = ratings.user_id.unique().shape[0]
number_of_movies = ratings.movie_id.unique().shape[0]
print(f"{number_of_users=}, {number_of_movies=}")

number_of_users=943, number_of_movies=1682


In [22]:
user_item_matrix=ratings.pivot_table(index=["movie_id"],columns=["user_id"],values="rating").reset_index(drop=True)
user_item_matrix.fillna(0,inplace=True)
user_item_matrix.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
0,5.0,4.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,4.0,...,2.0,3.0,4.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0
1,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,...,5.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
4,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
similarity_matrix=1-pairwise_distances(user_item_matrix,metric="correlation")
similarity_matrix


array([[ 1.        ,  0.23459453,  0.19336208, ..., -0.02967574,
         0.03417873,  0.03417873],
       [ 0.23459453,  1.        ,  0.1906486 , ..., -0.01245099,
         0.07141535,  0.07141535],
       [ 0.19336208,  0.1906486 ,  1.        , ..., -0.00976446,
        -0.00976446,  0.09142113],
       ...,
       [-0.02967574, -0.01245099, -0.00976446, ...,  1.        ,
        -0.00106157, -0.00106157],
       [ 0.03417873,  0.07141535, -0.00976446, ..., -0.00106157,
         1.        , -0.00106157],
       [ 0.03417873,  0.07141535,  0.09142113, ..., -0.00106157,
        -0.00106157,  1.        ]])

In [36]:
movie_name="Mission: Impossible (1996)"
movie_id=movies[movies["movie_name"]==movie_name].index
movie_id=movie_id[0]
movies["Similarity"]=similarity_matrix[movie_id]
movies.sort_values("Similarity",ascending=False)[1:11]


,movie_id,movie_name,Similarity
120,121,Independence Day (ID4) (1996),0.655003
596,597,Eraser (1996),0.577604
116,117,"Rock, The (1996)",0.567405
545,546,Broken Arrow (1996),0.543275
117,118,Twister (1996),0.542379
684,685,Executive Decision (1996),0.493904
221,222,Star Trek: First Contact (1996),0.437671
471,472,Dragonheart (1996),0.434790
741,742,Ransom (1996),0.430294
1046,1047,Multiplicity (1996),0.426702


In [5]:
data_matrix: ndarray = np.zeros((number_of_users, number_of_movies))
for line in ratings.itertuples():
    data_matrix[line[1] - 1, line[2] - 1] = line[3]
data_matrix_df = DataFrame(data_matrix)
data_matrix_df.head()

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
def predict(ratings: ndarray, similarity: ndarray, type: str = 'user') -> ndarray:
    pred = None
    # we are taking mean of axis=1 / rows
    # where axis0 -> y-axis and axis1 -> x-axis
    # i.e we are taking means of each row
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        # We use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array(
            [np.abs(similarity).sum(axis=1)]).T
    elif type == 'movie':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [7]:
user_similarity = pairwise_distances(data_matrix, metric='cosine')
movie_similarity = pairwise_distances(data_matrix.T, metric='cosine')

In [16]:
user_prediction = predict(data_matrix, user_similarity, type='user')
DataFrame(user_prediction).head()

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,2.065326,0.734303,0.629924,1.010669,0.640686,0.476150,1.784569,1.163032,1.513350,0.704478,...,0.394041,0.394434,0.393981,0.392972,0.393344,0.392272,0.394909,0.393590,0.393049,0.392771
1,1.763088,0.384040,0.196179,0.731538,0.225643,0.003892,1.493597,0.876153,1.108467,0.261991,...,-0.086942,-0.085491,-0.087137,-0.088158,-0.087298,-0.089288,-0.087468,-0.088378,-0.086918,-0.086712
2,1.795904,0.329047,0.158829,0.684154,0.173277,-0.035621,1.488230,0.835769,1.135426,0.236383,...,-0.134795,-0.133537,-0.135543,-0.136438,-0.135041,-0.137611,-0.136374,-0.136992,-0.134969,-0.134765
3,1.729951,0.293913,0.127741,0.644932,0.142143,-0.062261,1.437010,0.796249,1.096663,0.211789,...,-0.161413,-0.160220,-0.161542,-0.162586,-0.161634,-0.163877,-0.162283,-0.163080,-0.161442,-0.161248
4,1.796651,0.454474,0.354422,0.763130,0.359539,0.195987,1.547370,0.908904,1.292027,0.437954,...,0.101762,0.102405,0.101923,0.100839,0.101711,0.099951,0.102515,0.101233,0.101075,0.101201


In [12]:
movie_prediction = predict(data_matrix, movie_similarity, type="movie")
DataFrame(movie_prediction).head()

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,0.446278,0.475473,0.505938,0.443633,0.512667,0.547939,0.446243,0.463059,0.474916,0.515821,...,0.580579,0.576202,0.582478,0.582478,0.575717,0.588155,0.588155,0.588155,0.573107,0.566696
1,0.108544,0.132957,0.125589,0.124932,0.131178,0.129005,0.110883,0.122223,0.109599,0.121525,...,0.135490,0.136546,0.134829,0.134829,0.134108,0.134458,0.134458,0.134458,0.136576,0.137111
2,0.085685,0.091690,0.087643,0.089966,0.089658,0.089985,0.083492,0.089725,0.085188,0.088331,...,0.089770,0.090506,0.086261,0.086261,0.089201,0.084659,0.084659,0.084659,0.089768,0.090845
3,0.053693,0.059604,0.058114,0.058364,0.059356,0.061472,0.053374,0.058615,0.055905,0.060601,...,0.061349,0.061686,0.061195,0.061195,0.060693,0.057937,0.057937,0.057937,0.061673,0.062281
4,0.224739,0.229171,0.263280,0.226387,0.259973,0.296529,0.232710,0.237109,0.258581,0.275076,...,0.297628,0.295990,0.299922,0.299922,0.298188,0.302051,0.302051,0.302051,0.293373,0.294309
